## Imports

In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

In [2]:
!echo $CUDA_VISIBLE_DEVICES

2,3


In [3]:
DEVICE = torch.device('cuda')
DEVICE

device(type='cuda')

## Load model

In [4]:
%run ../models/checkpoint/__init__.py

In [5]:
# run_name = '1012_170550_cxr14_mobilenet_lr1e-06'
run_name = '0715_232418_cxr14_densenet-121_lr1e-06_os-max4_aug-0_Pneumonia'
debug_run = False

compiled_model = load_compiled_model_classification(run_name, debug=debug_run, device=DEVICE)
compiled_model.metadata

{'model_kwargs': {'model_name': 'densenet-121',
  'labels': ['Pneumonia'],
  'imagenet': True,
  'freeze': False},
 'opt_kwargs': {'lr': 1e-06},
 'hparams': {'loss_name': 'wbce', 'batch_size': 30},
 'dataset_kwargs': {'labels': ['Pneumonia'],
  'max_samples': None,
  'batch_size': 30},
 'dataset_train_kwargs': {'oversample': True,
  'oversample_label': 0,
  'oversample_max_ratio': 4,
  'augment': True,
  'augment_label': 0,
  'augment_kwargs': {'crop': 0.8,
   'translate': 0.1,
   'rotation': 15,
   'contrast': 0.5,
   'brightness': 0.5}}}

## Load data

In [6]:
%run ../datasets/__init__.py

In [7]:
kwargs = {
    'dataset_name': 'cxr14',
    'dataset_type': 'test-bbox',
    'max_samples': None,
    'batch_size': 10,
}

dataloader = prepare_data_classification(**kwargs)
len(dataloader.dataset)

Loading cxr14/test-bbox dataset...


880

## Run through data

In [8]:
import captum
from captum.attr import LayerGradCam
from torch import sigmoid
# from skimage.transform import resize
from torch.nn.functional import interpolate

In [9]:
%run ../utils/__init__.py

NOTE: Copied in eval_classification_grad_cam.py

In [40]:
iou = IoU(n_labels=len(dataloader.dataset.labels))
iou.reset()

for batch in iter(dataloader):
    ## Calculate bounding boxes
    images = batch.image.to(DEVICE)
    # shape: batch_size, 3, h, w

    bboxes_valid = batch.bboxes_valid.to(DEVICE) # shape: batch_size, n_labels
    bboxes = (batch.bboxes.to(DEVICE) / scale).long() # shape: batch_size, n_labels, 4
    bboxes_map = bbox_coordinates_to_map(bboxes, bboxes_valid, image_size)
    # shape: batch_size, n_labels, height, width
    
    ## Calculate attributions
    attributions = []
    for index, label in enumerate(dataloader.dataset.labels):
        attributions.append(calculate_attributions(images, index, image_size))
    attributions = torch.stack(attributions, dim=1)
    # shape: batch_size, n_labels, h, w
    
    attributions = threshold_attributions(attributions)
    # shape: batch_size, n_labels, h, w

    ## Calculate metrics
    iou.update((attributions, bboxes_map, bboxes_valid))

iou.compute()

tensor([0.0283, 0.1355, 0.0564, 0.0845, 0.0361, 0.0045, 0.0833, 0.0507, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000], device='cuda:0')